In [1]:
source('cascade_helper.R')


In [2]:
# p_thr <- list(1e-7, 1e-7, 5e-8)
p_thr <- list(5e-9, 5e-9, 5e-9)
names(p_thr) <- c('protein-truncating', 'protein-altering', 'non-coding')
p_thr_df <- as.data.frame(p_thr) %>% gather(Csq, p_thr) %>% mutate(Csq = str_replace_all(Csq, '\\.', '-'))


In [5]:
outd <- file.path(getwd(), 'out_v3')


In [6]:
annot.tbl <- '/oak/stanford/groups/mrivas/private_data/ukbb/variant_filtering/variant_filter_table.tsv.gz'
annot.arr <- read_annot_arr(annot.tbl)


In [7]:
annot.arr %>% count(Csq)


Csq,n
<chr>,<int>
non-coding,669033
protein-altering,87407
protein-truncating,27816


In [10]:
traits <- fread('cascade.input.files.tsv')


In [11]:
dim(traits)


[1] 35  4

In [12]:
array_anno_df <- traits %>% 
read_array_sumstats_all(p_thr_df %>% select(p_thr) %>% max()) %>%
annotate_array_df(annot.arr, p_thr_df)


In [13]:
array_anno_df %>%
dim()


[1] 79393    25

In [14]:
array_anno_df %>%
fwrite(file.path(outd, 'cascade.array.hits.tsv'), sep='\t')


In [15]:
array_anno_df %>% 
filter(Csq != 'non-coding') %>%
select(Csq, ld_indep, is_rare, is_outside_of_MHC, is_autosome, ID) %>%
unique() %>%
count(is_outside_of_MHC, Csq, ld_indep, is_rare, is_autosome)

is_outside_of_MHC,Csq,ld_indep,is_rare,is_autosome,n
<lgl>,<chr>,<lgl>,<lgl>,<lgl>,<int>
FALSE,protein-altering,FALSE,FALSE,TRUE,267
FALSE,protein-altering,FALSE,TRUE,TRUE,4
FALSE,protein-altering,TRUE,FALSE,TRUE,46
FALSE,protein-altering,TRUE,TRUE,TRUE,6
FALSE,protein-truncating,FALSE,FALSE,TRUE,15
FALSE,protein-truncating,FALSE,TRUE,TRUE,1
FALSE,protein-truncating,TRUE,TRUE,TRUE,1
TRUE,protein-altering,FALSE,FALSE,TRUE,1797
TRUE,protein-altering,FALSE,TRUE,TRUE,29


In [16]:
getwd()

[1] "/oak/stanford/groups/mrivas/users/ytanigaw/repos/rivas-lab/biomarkers/cascade"

```
ml load gdrive
gdrive upload -p 1fGlnrj8Zu1Pox3fmxmFKdcbNVWiN9vhd out_v3/cascade.array.hits.tsv
zstd -15 --rm out_v3/cascade.array.hits.tsv
```